In [ ]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import numpy as np



In [ ]:
from keras import applications
def vgg16Model(image_size = (224, 224), if_draw_model = False):
    width = image_size[0]
    height = image_size[1]
    
    base_model = applications.VGG16(input_tensor=Input((height, width, 3)), weights='imagenet', include_top=False)

    for layers in base_model.layers:
        layers.trainable = True
        
    for i, layer in enumerate(base_model.layers):
        print(i, layer.name)
        
   # for layer in base_model.layers[25:]:
   #     layer.trainable = True

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(10, activation='sigmoid')(x)
    model = Model(base_model.input, x)
    
    if if_draw_model:
        SVG(model_to_dot(model).create(prog='dot', format='svg'))
    
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

def restNet_model(image_size = (224, 224), if_draw_model = False):

    width = image_size[0]
    height = image_size[1]
    
    base_model = ResNet50(input_tensor=Input((height, width, 3)), weights='imagenet', include_top=False)

    for layers in base_model.layers:
        layers.trainable = True
        
    for i, layer in enumerate(base_model.layers):
        print(i, layer.name)
        
   # for layer in base_model.layers[25:]:
   #     layer.trainable = True

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(10, activation='sigmoid')(x)
    model = Model(base_model.input, x)
    
    if if_draw_model:
        SVG(model_to_dot(model).create(prog='dot', format='svg'))
    
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input, decode_predictions
import numpy as np

def inceptionV3_model(image_size = (224, 224), if_draw_model = False):

    width = image_size[0]
    height = image_size[1]
    
    base_model = InceptionV3(input_tensor=Input((height, width, 3)), weights='imagenet', include_top=False)

    for layers in base_model.layers:
        layers.trainable = True
        
    for i, layer in enumerate(base_model.layers):
        print(i, layer.name)
        
   # for layer in base_model.layers[25:]:
   #     layer.trainable = True

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(10, activation='sigmoid')(x)
    model = Model(base_model.input, x)
    
    if if_draw_model:
        SVG(model_to_dot(model).create(prog='dot', format='svg'))
    
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [ ]:
import os
import shutil
from sklearn.utils import shuffle
np.random.seed(2017)

def prepare_val():
    dirname = 'imgs/val'
    dirname_src = 'imgs/train/'
    if os.path.exists(dirname):
        return
    os.mkdir(dirname)
    sub_dirs = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
    for sub_dir in sub_dirs:
        address_des = dirname + '/' + sub_dir
        os.mkdir(address_des)
        
        address_src = dirname_src + '/' + sub_dir
        train_filenames = os.listdir(address_src)
        train_filenames = shuffle(train_filenames)
        for file_src in train_filenames[0: int(len(train_filenames) / 10)]:
            add_con_list = file_src.split('/')
            add_old = dirname_src + sub_dir + '/' + add_con_list[-1]
            add_new = dirname + '/' + sub_dir + '/' + add_con_list[-1]
         #   print(add_old)
         #   print(add_new)
            shutil.move(add_old, add_new)  
        
prepare_val()

In [ ]:
image_size=(224, 224)
batch_size =64

In [ ]:
#训练
from keras.callbacks import EarlyStopping
#from keras.applications.vgg19 import preprocess_input
#from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.applications.inception_v3 import preprocess_input, decode_predictions

nb_train_samples = 20197
def train_model(Model):
    model = Model()
    
    train_datagen = ImageDataGenerator(
 #   shear_range=0.2,
 #   zoom_range=0.2,
    preprocessing_function=preprocess_input,
 #   horizontal_flip=True
    )
    
    train_generator = train_datagen.flow_from_directory(
    'imgs/train',
    target_size=(image_size[0], image_size[1]),
    batch_size=batch_size,
    class_mode='categorical')
    
    val_generator = train_datagen.flow_from_directory(
    'imgs/val',
    target_size=(image_size[0], image_size[1]),
    batch_size=batch_size,
    class_mode='categorical')
        
    model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=15,
    workers=4,
    callbacks=[EarlyStopping(monitor='val_loss', patience=5)],
    validation_data = val_generator)
    
  #  model.save_weights('model.h5')
    
    return model


In [ ]:
import pandas as pd
import cv2
def test_model(model, file_name):
    df = pd.read_csv("sample_submission.csv")
    
    test_address = 'imgs/test'
    test_filenames = os.listdir(test_address)
 #   test_filenames = test_filenames[:10]
    X = np.zeros((len(test_filenames), image_size[0], image_size[1], 3), dtype=np.uint8)

    for i, test_file in enumerate(test_filenames) :
        X[i] = cv2.resize(cv2.imread(test_address + '/' + test_file), image_size)
    
    y_pred = model.predict(X, batch_size=batch_size, verbose=0)
  #  print(y_pred)
    
    for i, fname in enumerate(test_filenames):
      #  print(fname)
        df.set_value(i, 'img', fname)
        sub_dirs = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
        for ii, sb in enumerate(sub_dirs):
            df.set_value(i, sb, y_pred[i][ii])


    df.to_csv(file_name, index=None)
    df.head(10)
    

In [ ]:
import pandas as pd
import cv2
def test2_model(model, file_name):
    df = pd.read_csv("sample_submission.csv")
    
    test_address = 'imgs/test'
    test_filenames = os.listdir(test_address)
    gen = ImageDataGenerator()
    test_generator = gen.flow_from_directory(test_address, image_size, shuffle=False, 
                                             batch_size=len(test_filenames), class_mode='categorical')

    y_pred = model.predict_generator(test_generator, test_generator.samples)
    
    print(y_pred)
    
    for i, fname in enumerate(test_filenames):
      #  print(fname)
        df.set_value(i, 'img', fname)
        sub_dirs = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
        for ii, sb in enumerate(sub_dirs):
            df.set_value(i, sb, y_pred[i][ii])


    df.to_csv(file_name, index=None)
    df.head(10)

In [ ]:
model = train_model(restNet_model)